# Visualyzing Marconi100's `mem_free` job profiles

## Read Data

Here i visualize only the single-node jobs `/plugin=job_table/metric=job_info_marconi100/metric=mem_free_filter123_singlenode.csv`. We need to modify the plotting script to vizualize for multiple nodes (same path as but with a `multinode` suffix)

IMPORTANT: To make it easier to understand the plots. I'm converting the `mem_free` to its corresponding consumed memory values (i.e., 256GB - `mem_free`). This way is easier to spot measurement errors (i.e., negative values)
Some jobs have "nice" curves but with negative values

In [1]:
import pandas as pd

GIGABYTES_ONE_KILOBYTE = 9.5367431640625e-7

M100_MAX_MEM_GB = 256 

df_jobs_viz = pd.read_csv("../example_data/plugin=job_table/metric=job_info_marconi100/metric=mem_free_filter123_singlenode.csv")

df_metric = pd.read_parquet("../example_data/plugin=ganglia_pub/metric=mem_free/a_0.parquet")
df_metric['node'] = pd.to_numeric(df_metric['node'])
df_metric['value'] = pd.to_numeric(df_metric['value'])

#Converting KB to GB
df_metric['value'] = df_metric['value'] * GIGABYTES_ONE_KILOBYTE

# Comment the line below if you want to visualize the mem_free and not the consumed memory
df_metric['value'] = M100_MAX_MEM_GB - df_metric['value']
df_metric['value'].describe()

count    5.338966e+07
mean     3.730951e+01
std      3.801913e+01
min     -5.257635e+01
25%      1.047766e+01
50%      3.227509e+01
75%      6.160571e+01
max      2.556728e+02
Name: value, dtype: float64

## Plotting the `max_memory` profile for each job (interactive)

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
import matplotlib.pyplot as plt
import ast

#gfig = None

def call_workflow(index):
    get_job_energy_profile(index)

def get_job_energy_profile(index):
    global current_job_id, current_hostname
    annot_str=''
    job=df_jobs_viz.iloc[index,:]
    nodes=ast.literal_eval(job['nodes'])
    start_time=job["start_time"]
    end_time=job["end_time"]

    current_job_id=str(job['job_id'])
    for node in nodes:
        #print(node)
        df_node = df_metric.loc[df_metric['node'] == node]
        df_node_job = df_node.loc[df_node['timestamp'].between(start_time, end_time)]
        if len(df_node_job) > 0:
            #print(df_node_job.describe(), end="\r")
            plot_energy_profile(df_node_job, job, annot_str)
    #print(nodes)
     
    #plot_energy_profile(job_energy_profile, annot_str)    
    #current_hostname=energy_host
    ##
    #describe=job_energy_profile.describe(percentiles=[.10, .25, .5, .75, .90])
    #describe['job_id']=job['job_id']
    #describe['socket']=socket
    #describe['pp0']=describe[right_pp0]
    #describe['DRAM']=describe[right_DRAM]
    #describe['stat']=describe.index
    #describe=describe.reset_index(drop=True)[arr_cols]  
    #print(describe)  


def plot_energy_profile(profile, job, annot_str):
    TINY_SIZE = 2
    SMALL_SIZE = 5
    MEDIUM_SIZE = 25
    BIGGER_SIZE = 50
    FIG_WIDTH = 40
    FIG_HEIGHT = 10

    global gfig, gjob, gprofile  

    gjob=job
    gprofile=profile

    plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)     # fontsize of the x and y labels
    plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
    scatterplot_kwargs={'s': 50, 'palette': 'plasma'}
    lineplot_kwargs={'linewidth': 1}

    plt.clf()
    fig = plt.figure(figsize=(FIG_WIDTH,FIG_HEIGHT))
    #ax = sns.boxplot(x='stat', y='value', data=plot_data, showfliers=False, hue='reading',
    #             linewidth=TINY_SIZE)

    #ax = sns.scatterplot(data=profile, x='timestamp', y='value', **scatterplot_kwargs)
    ax = sns.lineplot(data=profile, x='timestamp', y='value', **lineplot_kwargs)

    ## SET BORDERS SIZE AND WIDTH
    [line.set_linewidth(TINY_SIZE) for line in ax.spines.values()]
    [line.set_markersize(TINY_SIZE) for line in ax.yaxis.get_ticklines()]
    [line.set_markeredgewidth(TINY_SIZE) for line in ax.yaxis.get_ticklines()]
    [line.set_markersize(SMALL_SIZE) for line in ax.xaxis.get_ticklines()]
    [line.set_markeredgewidth(TINY_SIZE) for line in ax.xaxis.get_ticklines()]
    #ax.text(x=0.1,y=0.5,
    #        s=annot_str,
    #        fontdict=dict(color='red',size=MEDIUM_SIZE),
    #        bbox=dict(facecolor='yellow',alpha=0.5),
    #        horizontalalignment='left',
    #        verticalalignment='center',
    #        transform=ax.transAxes)
    ax.set_ylabel('Node used memory (GB)')
    ax.set_xlabel('Timestamp')
    ax.set_title('Job ID: '+current_job_id)    
    gfig = fig

button_saveplot = widgets.Button(description="Save to PDF")
button_savejobinfo = widgets.Button(description="Save job info to CSV")
button_savememinfo = widgets.Button(description="Save mem. profile to CSV")
output = widgets.Output()

display(button_saveplot, button_savejobinfo, button_savememinfo, output)
#display(button_saveplot, output)

## TODO: Pass on b the data (job_id, hostname, etc)
def save_plot_to_pdf_click(b):
    with output:
        #fig=plt.gcf()
        #print(gfig)
        fig_filename='../figures/marconi100_interact_plot_mem_free_profile_'+current_job_id+'.pdf'
        gfig.savefig(fig_filename, format='pdf', dpi=300, bbox_inches='tight')
        print('Plot saved as '+fig_filename, end="\r")

def save_job_info_click(b):
    with output:
        job_filename='../samples_jobs_mem_profiles/'+current_job_id+'_job_info.csv'
        pd.DataFrame(gjob).transpose().to_csv(job_filename, index=False, sep=",")        
        print('Job info saved as '+job_filename, end="\r")

def save_mem_profile_click(b):
    with output:
        profile_filename='../samples_jobs_mem_profiles/'+current_job_id+'_mem_profile.csv'
        gprofile.to_csv(profile_filename, index=False, sep=",")
        print('Job mem profile saved as '+profile_filename, end="\r")

button_saveplot.on_click(save_plot_to_pdf_click)
button_savejobinfo.on_click(save_job_info_click)
button_savememinfo.on_click(save_mem_profile_click)

interact(call_workflow, index=widgets.IntSlider(min=0, max=len(df_jobs_viz)-1, step=1, value=0));

Button(description='Save to PDF', style=ButtonStyle())

Button(description='Save job info to CSV', style=ButtonStyle())

Button(description='Save mem. profile to CSV', style=ButtonStyle())

Output()

interactive(children=(IntSlider(value=0, description='index', max=350), Output()), _dom_classes=('widget-inter…

# Visualization per user

In [16]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
import matplotlib.pyplot as plt
import ast

#gfig = None



def call_workflow(index,user_id):
    get_job_energy_profile(index, user_id)

def get_job_energy_profile(index, user_id):
    global current_job_id, current_hostname, user_nb_jobs
    annot_str=''
    df_jobs_viz_user = df_jobs_viz.loc[df_jobs_viz["user_id"] == user_id]
    user_nb_jobs = len(df_jobs_viz_user)
    job=df_jobs_viz_user.iloc[index,:]
    nodes=ast.literal_eval(job['nodes'])
    start_time=job["start_time"]
    end_time=job["end_time"]

    current_job_id=str(job['job_id'])
    for node in nodes:
        #print(node)
        df_node = df_metric.loc[df_metric['node'] == node]
        df_node_job = df_node.loc[df_node['timestamp'].between(start_time, end_time)]
        if len(df_node_job) > 0:
            #print(df_node_job.describe(), end="\r")
            plot_energy_profile(df_node_job, job, annot_str)
    #print(nodes)
     
    #plot_energy_profile(job_energy_profile, annot_str)    
    #current_hostname=energy_host
    ##
    #describe=job_energy_profile.describe(percentiles=[.10, .25, .5, .75, .90])
    #describe['job_id']=job['job_id']
    #describe['socket']=socket
    #describe['pp0']=describe[right_pp0]
    #describe['DRAM']=describe[right_DRAM]
    #describe['stat']=describe.index
    #describe=describe.reset_index(drop=True)[arr_cols]  
    #print(describe)  


def plot_energy_profile(profile, job, annot_str):
    TINY_SIZE = 2
    SMALL_SIZE = 5
    MEDIUM_SIZE = 25
    BIGGER_SIZE = 50
    FIG_WIDTH = 40
    FIG_HEIGHT = 10

    global gfig, gjob, gprofile  

    gjob=job
    gprofile=profile

    plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)     # fontsize of the x and y labels
    plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
    scatterplot_kwargs={'s': 50, 'palette': 'plasma'}
    lineplot_kwargs={'linewidth': 1}

    plt.clf()
    fig = plt.figure(figsize=(FIG_WIDTH,FIG_HEIGHT))
    #ax = sns.boxplot(x='stat', y='value', data=plot_data, showfliers=False, hue='reading',
    #             linewidth=TINY_SIZE)

    #ax = sns.scatterplot(data=profile, x='timestamp', y='value', **scatterplot_kwargs)
    ax = sns.lineplot(data=profile, x='timestamp', y='value', **lineplot_kwargs)

    ## SET BORDERS SIZE AND WIDTH
    [line.set_linewidth(TINY_SIZE) for line in ax.spines.values()]
    [line.set_markersize(TINY_SIZE) for line in ax.yaxis.get_ticklines()]
    [line.set_markeredgewidth(TINY_SIZE) for line in ax.yaxis.get_ticklines()]
    [line.set_markersize(SMALL_SIZE) for line in ax.xaxis.get_ticklines()]
    [line.set_markeredgewidth(TINY_SIZE) for line in ax.xaxis.get_ticklines()]
    #ax.text(x=0.1,y=0.5,
    #        s=annot_str,
    #        fontdict=dict(color='red',size=MEDIUM_SIZE),
    #        bbox=dict(facecolor='yellow',alpha=0.5),
    #        horizontalalignment='left',
    #        verticalalignment='center',
    #        transform=ax.transAxes)
    ax.set_ylabel('Node used memory (GB)')
    ax.set_xlabel('Timestamp')
    ax.set_title('Job ID: '+current_job_id)    
    gfig = fig

button_saveplot = widgets.Button(description="Save to PDF")
button_savejobinfo = widgets.Button(description="Save job info to CSV")
button_savememinfo = widgets.Button(description="Save mem. profile to CSV")
output = widgets.Output()

display(button_saveplot, button_savejobinfo, button_savememinfo, output)
#display(button_saveplot, output)

## TODO: Pass on b the data (job_id, hostname, etc)
def save_plot_to_pdf_click(b):
    with output:
        #fig=plt.gcf()
        #print(gfig)
        fig_filename='../figures/marconi100_interact_plot_mem_free_profile_'+current_job_id+'.pdf'
        gfig.savefig(fig_filename, format='pdf', dpi=300, bbox_inches='tight')
        print('Plot saved as '+fig_filename, end="\r")

def save_job_info_click(b):
    with output:
        job_filename='../samples_jobs_mem_profiles/'+current_job_id+'_job_info.csv'
        pd.DataFrame(gjob).transpose().to_csv(job_filename, index=False, sep=",")        
        print('Job info saved as '+job_filename, end="\r")

def save_mem_profile_click(b):
    with output:
        profile_filename='../samples_jobs_mem_profiles/'+current_job_id+'_mem_profile.csv'
        gprofile.to_csv(profile_filename, index=False, sep=",")
        print('Job mem profile saved as '+profile_filename, end="\r")

button_saveplot.on_click(save_plot_to_pdf_click)
button_savejobinfo.on_click(save_job_info_click)
button_savememinfo.on_click(save_mem_profile_click)

user_ids = df_jobs_viz["user_id"].drop_duplicates().values.tolist()

drp_userids = widgets.Dropdown(
    options=user_ids,
    value=user_ids[0],
    description='User ID: ',
    disabled=False,
)

print(len(df_jobs_viz))
user_nb_jobs = len(df_jobs_viz.loc[df_jobs_viz["user_id"]==user_ids[0]])

slider_index_jobs = widgets.IntSlider(min=0, max=user_nb_jobs-1, step=1, value=0)

#drp_userids.observe(get_job_energy_profile)

# Define a function that updates the intslider's range based on the dropdown selection
def update_range(*args):
    # Update the range of the intslider based on the selected option in the dropdown
    user_nb_jobs = len(df_jobs_viz.loc[df_jobs_viz["user_id"]==drp_userids.value])    
    slider_index_jobs.min = 0
    slider_index_jobs.max = user_nb_jobs-1
    # Reset the intslider's value to the start of the new range
    slider_index_jobs.value = slider_index_jobs.min

drp_userids.observe(update_range, 'value')


interact(call_workflow, index=slider_index_jobs, user_id = drp_userids);

Button(description='Save to PDF', style=ButtonStyle())

Button(description='Save job info to CSV', style=ButtonStyle())

Button(description='Save mem. profile to CSV', style=ButtonStyle())

Output()

351


interactive(children=(IntSlider(value=0, description='index', max=53), Dropdown(description='User ID: ', optio…

Users with jobs with similar memory profile

- 1234: a lot of constant and step-wise jobs
- 1702: memory consumption increases over time and stays until the end
- 1436: very short jobs (~2 min long)
- 2: similar patten on all jobs 
- 1167: same pattern
- 1474: multiple patterns but similar

# Multi-Node jobs

In [28]:
import pandas as pd
from datetime import datetime

GIGABYTES_ONE_KILOBYTE = 9.5367431640625e-7

#M100_MAX_MEM_GB = 256

# Virtually increasing the max memory of M100
# This is may be due to the data collection
# The max free memory can be more than 256GB (308.57635498046875)
# Even though the doc says (RAM: 256 GB/node (242 usable))
# Setting to 320 is ok if we also set the simulation to 320GB 
# The job memory profile still holds
M100_MAX_MEM_GB = 320

NANOSECONDS_ONE_SECOND = 1e9

df_jobs_viz_multi = pd.read_csv("../example_data/plugin=job_table/metric=job_info_marconi100/metric=mem_free_filter123_multinode.csv")

df_metric = pd.read_parquet("../example_data/plugin=ganglia_pub/metric=mem_free/a_0.parquet")
df_metric['node'] = pd.to_numeric(df_metric['node'])
df_metric['value'] = pd.to_numeric(df_metric['value'])

#Converting KB to GB
df_metric['value'] = df_metric['value'] * GIGABYTES_ONE_KILOBYTE

print(df_metric['value'].max())

# Comment the line below if you want to visualize the mem_free and not the consumed memory
df_metric['value'] = M100_MAX_MEM_GB - df_metric['value']

# Adding the timestamp in seconds (need to fix this)
#df_metric["timestamp_seconds"] = pd.to_datetime(df_metric['timestamp']).astype(int) / NANOSECONDS_ONE_SECOND
df_metric["timestamp_seconds"] = [datetime.timestamp(t) for t in df_metric['timestamp'].values.tolist()]

print(df_metric.head(10))

308.57635498046875
            timestamp      value  node  timestamp_seconds
0 2022-05-22 13:45:08  34.138245     5       1.653220e+09
1 2022-05-22 13:46:28  34.126587     5       1.653220e+09
2 2022-05-22 13:47:08  34.134155     5       1.653220e+09
3 2022-05-22 13:47:49  34.136963     5       1.653220e+09
4 2022-05-22 13:49:48  34.139160     5       1.653220e+09
5 2022-05-22 13:50:28  34.140137     5       1.653220e+09
6 2022-05-22 13:51:08  34.141663     5       1.653220e+09
7 2022-05-22 13:52:28  34.143738     5       1.653220e+09
8 2022-05-22 13:53:08  34.141418     5       1.653220e+09
9 2022-05-22 13:54:28  34.146118     5       1.653220e+09


In [38]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
import scipy.stats as st
import ipywidgets as widgets
import seaborn as sns
import matplotlib.pyplot as plt
import ast

#gfig = None
TINY_SIZE = 2
SMALL_SIZE = 5
MEDIUM_SIZE = 25
BIGGER_SIZE = 50
FIG_WIDTH = 40
FIG_HEIGHT = 10

ZCORE_EXTREME_OUTLIERS = 3
ZSCORE_UNUSUAL_OUTLIERS = 2

global_job_mem_phases = None
PHASES_COLS = ["user_id", "job_id", "node", "start_timestamp", "end_timestamp", "max_memory"]

def call_workflow(index,user_id):
    get_job_energy_profile(index, user_id)

def get_profile_phases(profile, job):
    global global_job_mem_phases
    plt.clf()
    fig = plt.figure(figsize=(FIG_WIDTH,FIG_HEIGHT))
    groups = profile.groupby("node")
    nodes = profile["node"].drop_duplicates().to_list()
    job_phases = []
    for i, node in enumerate(nodes):
        g = groups.get_group(node).copy().reset_index()
        g['timestamp_diff'] = g.sort_values("timestamp_seconds")["value"].diff()
        #g = g.dropna(subset='timestamp_diff')
        g = g.fillna(0.0)
        g["zscore"] = st.zscore(g['timestamp_diff'].values)
        #print(g["zscore"])
        phases = g.loc[g["zscore"].abs() > ZSCORE_UNUSUAL_OUTLIERS]
        phases = pd.concat([g.head(1), phases, g.tail(1)]).reset_index()
        for index, row in phases.iterrows():
            if(index+1 < len(phases)):
                start = row
                end = phases.iloc[index+1]
                window = g.loc[g["timestamp_seconds"].between(start["timestamp_seconds"], end["timestamp_seconds"])]["value"]                
                ## dropping the last element to calculate the border height
                window = window.drop(window.tail(1).index, inplace=False)
                mem_max = window.max()
                phase_length = end["timestamp_seconds"] - start["timestamp_seconds"]
                #print(start["timestamp_seconds"], end["timestamp_seconds"], mem_max)
                job_phases.append([job["user_id"], job["job_id"], node, start["timestamp"], end["timestamp"], mem_max])
                ## Only plot for the first node
                if i == 0:
                    xy = (start["timestamp_seconds"],0)
                    width = phase_length
                    height = mem_max
                    #print(xy, width, height)
                    rct = Rectangle(xy=xy, width=width, height=height, facecolor=None, linestyle="-", edgecolor="r", linewidth=5, fill=False)
                    plt.gca().add_patch(rct)
                #rectangles.append(rct)
                #plt.axhline(y=mem_max, xmin=start["timestamp_seconds"], xmax=end["timestamp_seconds"])
                #plt.axhline(y=10, xmin=start["timestamp_seconds"]/phase_length, xmax=end["timestamp_seconds"]/phase_length)
                #print(phase_length)
                #print(start["timestamp_seconds"]/phase_length, end["timestamp_seconds"]/phase_length)

        #pc = PatchCollection(rectangles, edgecolor="b")
        #plt.gca().add_collection(pc)
        #print(phases["timestamp_seconds"].values.tolist())       
        #plt.vlines(x=phases["timestamp_seconds"].values.tolist(),ymin=profile["value"].min(), ymax=profile["value"].max())
        #print(phases["timestamp"].values.tolist())
        #break
    df_job_phases = pd.DataFrame(data=job_phases, columns=PHASES_COLS)
    global_job_mem_phases = df_job_phases
    print(df_job_phases)
       

def get_job_energy_profile(index, user_id):
    global current_job_id, current_hostname, user_nb_jobs
    annot_str=''
    df_jobs_viz_user = df_jobs_viz_multi.loc[df_jobs_viz_multi["user_id"] == user_id]
    user_nb_jobs = len(df_jobs_viz_user)
    job=df_jobs_viz_user.iloc[index,:]
    nodes=ast.literal_eval(job['nodes'])
    start_time=job["start_time"]
    end_time=job["end_time"]

    current_job_id=str(job['job_id'])
    #for node in nodes:
        #print(node)
    df_node = df_metric.loc[df_metric['node'].isin(nodes)]
    df_node_job = df_node.loc[df_node['timestamp'].between(start_time, end_time)]
    if len(df_node_job) > 0:
        #print(df_node_job.describe(), end="\r")
        get_profile_phases(df_node_job, job)        
        plot_energy_profile(df_node_job, job, annot_str)
    #print(nodes)
     
    #plot_energy_profile(job_energy_profile, annot_str)    
    #current_hostname=energy_host
    ##
    #describe=job_energy_profile.describe(percentiles=[.10, .25, .5, .75, .90])
    #describe['job_id']=job['job_id']
    #describe['socket']=socket
    #describe['pp0']=describe[right_pp0]
    #describe['DRAM']=describe[right_DRAM]
    #describe['stat']=describe.index
    #describe=describe.reset_index(drop=True)[arr_cols]  
    #print(describe)  


def plot_energy_profile(profile, job, annot_str):
    global gfig, gjob, gprofile  

    gjob=job
    gprofile=profile

    plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)     # fontsize of the x and y labels
    plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
    scatterplot_kwargs={'s': 50, 'palette': 'plasma'}
    lineplot_kwargs={'linewidth': 1}

    #plt.clf()
    #fig = plt.figure(figsize=(FIG_WIDTH,FIG_HEIGHT))


    #ax = sns.boxplot(x='stat', y='value', data=plot_data, showfliers=False, hue='reading',
    #             linewidth=TINY_SIZE)

    #ax = sns.scatterplot(data=profile, x='timestamp', y='value', **scatterplot_kwargs)

    palette = sns.color_palette("rocket")
    #ax = sns.lineplot(data=profile, x='timestamp_seconds', y='value', hue='node', palette=palette, **lineplot_kwargs)
    ax = sns.scatterplot(data=profile, x='timestamp_seconds', y='value', hue='node', palette=palette, s=10)

    ## SET BORDERS SIZE AND WIDTH
    [line.set_linewidth(TINY_SIZE) for line in ax.spines.values()]
    [line.set_markersize(TINY_SIZE) for line in ax.yaxis.get_ticklines()]
    [line.set_markeredgewidth(TINY_SIZE) for line in ax.yaxis.get_ticklines()]
    [line.set_markersize(SMALL_SIZE) for line in ax.xaxis.get_ticklines()]
    [line.set_markeredgewidth(TINY_SIZE) for line in ax.xaxis.get_ticklines()]
    #ax.text(x=0.1,y=0.5,
    #        s=annot_str,
    #        fontdict=dict(color='red',size=MEDIUM_SIZE),
    #        bbox=dict(facecolor='yellow',alpha=0.5),
    #        horizontalalignment='left',
    #        verticalalignment='center',
    #        transform=ax.transAxes)
    ax.set_ylabel('Node used memory (GB)')
    ax.set_xlabel('Timestamp')
    ax.set_title('Job ID: '+current_job_id)    
    gfig = ax.get_figure()

button_saveplot = widgets.Button(description="Save to PDF")
button_savejobinfo = widgets.Button(description="Save job info to CSV")
button_savememinfo = widgets.Button(description="Save mem. profile to CSV")
output = widgets.Output()

display(button_saveplot, button_savejobinfo, button_savememinfo, output)
#display(button_saveplot, output)

## TODO: Pass on b the data (job_id, hostname, etc)
def save_plot_to_pdf_click(b):
    with output:
        #fig=plt.gcf()
        #print(gfig)
        fig_filename='../figures/marconi100_interact_plot_mem_free_profile_'+current_job_id+'.pdf'
        gfig.savefig(fig_filename, format='pdf', dpi=300, bbox_inches='tight')
        print('Plot saved as '+fig_filename, end="\r")

def save_job_info_click(b):
    with output:
        job_filename='../samples_jobs_mem_profiles/'+current_job_id+'_job_info.csv'
        pd.DataFrame(gjob).transpose().to_csv(job_filename, index=False, sep=",")        
        print('Job info saved as '+job_filename, end="\r")

def save_mem_profile_click(b):
    with output:
        profile_filename='../samples_jobs_mem_profiles/'+current_job_id+'_mem_profile.csv'
        gprofile.to_csv(profile_filename, index=False, sep=",")
        phases_profile_filename='../samples_jobs_mem_profiles/'+"user_"+str(global_job_mem_phases["user_id"].values[0])+"_job_"+str(global_job_mem_phases["job_id"].values[0])+'_mem_profile.csv'
        global_job_mem_phases.to_csv(phases_profile_filename, index=False, sep=",")
        print('Job mem profile saved as '+profile_filename, end="\t")
        print('Job mem profile phases saved as '+phases_profile_filename, end="\r")

button_saveplot.on_click(save_plot_to_pdf_click)
button_savejobinfo.on_click(save_job_info_click)
button_savememinfo.on_click(save_mem_profile_click)

user_ids = df_jobs_viz_multi["user_id"].drop_duplicates().values.tolist()

drp_userids = widgets.Dropdown(
    options=user_ids,
    value=user_ids[0],
    description='User ID: ',
    disabled=False,
)

print(len(df_jobs_viz_multi))
user_nb_jobs = len(df_jobs_viz_multi.loc[df_jobs_viz_multi["user_id"]==user_ids[0]])

slider_index_jobs = widgets.IntSlider(min=0, max=user_nb_jobs-1, step=1, value=0)

#drp_userids.observe(get_job_energy_profile)

# Define a function that updates the intslider's range based on the dropdown selection
def update_range(*args):
    # Update the range of the intslider based on the selected option in the dropdown
    user_nb_jobs = len(df_jobs_viz_multi.loc[df_jobs_viz_multi["user_id"]==drp_userids.value])    
    slider_index_jobs.min = 0
    slider_index_jobs.max = user_nb_jobs-1
    # Reset the intslider's value to the start of the new range
    slider_index_jobs.value = slider_index_jobs.min

drp_userids.observe(update_range, 'value')


interact(call_workflow, index=slider_index_jobs, user_id = drp_userids);

Button(description='Save to PDF', style=ButtonStyle())

Button(description='Save job info to CSV', style=ButtonStyle())

Button(description='Save mem. profile to CSV', style=ButtonStyle())

Output()

966


interactive(children=(IntSlider(value=0, description='index', max=9), Dropdown(description='User ID: ', option…

Users: 

- 1430: long jobs
- 790
- 521: samme pattern
- 632: memory leak
- 88: some synchronized phases
- 838: some syncrhonized phases
- 977: same pattern across nodes, periodicity with a large mem consumption
- 96: a very weird app
- 184: very long jobs
- 1427: something close to the "all over the place"
- 1681: a mix of periodic and constant
- 640: two-phase job


In [3]:
df_metric.describe(percentiles=[0.25,0.5,0.75,0.9])
#sns.histplot(data=df_metric, x="value")

,timestamp,value,node
count,53389664,5.338966e+07,5.338966e+07
mean,2022-05-16 09:48:45.024000,3.730951e+01,4.955961e+02
min,2022-05-01 00:00:00,-5.257635e+01,0.000000e+00
25%,2022-05-08 11:37:50,1.047766e+01,2.480000e+02
50%,2022-05-16 10:50:43,3.227509e+01,4.950000e+02
75%,2022-05-24 05:46:40,6.160571e+01,7.440000e+02
90%,2022-05-28 21:13:44,8.632166e+01,8.910000e+02
max,2022-05-31 23:59:59,2.556728e+02,9.870000e+02
std,NaN,3.801913e+01,2.858347e+02
